In [1]:
from tqdm.notebook import tqdm
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.alert import Alert

# 1. kobis 에서 역대 박스오피스 크롤링

In [ ]:
# FULLXPATH
url = 'https://www.kobis.or.kr/kobis/business/stat/boxs/findFormerBoxOfficeList.do'
driver=webdriver.Chrome(service=Service(ChromeDriverManager().install()))
# driver.get(url)

def wait_for_element(element_xpath, timeout=100):
    try:
        element = WebDriverWait(driver, timeout).until(
            EC.presence_of_element_located((By.XPATH, element_xpath))
        )
        return element
    except Exception as e:
        print(f"Error: {e}")
        return None
    
try:

    for i in range(41):
        driver.get(url)
        movie_element = driver.find_element(By.ID, 'tr_{}'.format(i)) # ok

        if movie_element:
            movie_element.find_element(By.CLASS_NAME, 'ellip').click()
            info_xpath = '/html/body/div[2]/div[1]/div[2]/ul/li[2]/a'
            info_element = wait_for_element(info_xpath) # 통계정보 기다리기

            if info_element:
                info_element.click() # 통계정보 누르기 ------------fullXPATH로 변경.
                exel_xpath = '/html/body/div[2]/div[2]/div/div[2]/div[3]/div/div/div/div/a'                        
                exel_element = wait_for_element(exel_xpath) #엑셀 기다리기

                if exel_element:
                    exel_element.click() #엑셀 누르기
                    # Handle the '확인' alert
                    da = Alert(driver)
                    da.accept()

except Exception as e:
    print(f"Error: {e}")

# 2. 엑셀파일 xls -> xlsx 변환

In [ ]:
import os
import win32com.client as win32
#pip install pypiwin32


# 엑셀 변환 함수
def convert_xls_to_xlsx(xls_file_path, xlsx_file_path):
    excel = win32.Dispatch("Excel.Application")
    excel.DisplayAlerts = False
    wb = excel.Workbooks.Open(xls_file_path)
    wb.SaveAs(xlsx_file_path, FileFormat=51)  # FileFormat=51 corresponds to .xlsx
    wb.Close()
    excel.Quit()

    
input_folder = 'C:\\Users\\acorn\\Downloads'
output_folder = 'C:\\xlsx'

# 반복
for file_name in os.listdir(input_folder):
    if file_name.endswith('.xls'):
        input_file_path = os.path.join(input_folder, file_name)
        output_file_name = file_name.replace('.xls', '.xlsx')
        output_file_path = os.path.join(output_folder, output_file_name)

        print(f"Converting {input_file_path} to {output_file_path}")
        convert_xls_to_xlsx(input_file_path, output_file_path)
        print(f"Conversion complete for {output_file_path}")


# 3. 개봉일전인 시사회 삭제, 스크린 점유율이 1% 미만일시 상영종료

In [ ]:
movie_dfs1=[]
movie_dfs2=[] #영화 상영 종료일로 잘라서 재저장. 리스트

# ==========================================================

# 영화 제목으로 저장하기위해 역대영화 리스트 가져오기.   
movie_info = pd.read_csv('movie_info_200.csv')
movie_info   

print('영화 제목으로 저장하기위해 역대영화 리스트 가져오기. 완')


# ==========================================================



# 상영 종료일은 최근 5일간 상영횟수가 5이하일경우일때 상영횟수 카운팅 시작한 날짜.
def movie_end(df):
    day_index=0
    for j in range(len(df.index)):
        if df.iloc[j,2]*100 < 1.0: # 스크린 점유율이 1미만 이면 종료.
            
            day_index = j
            print(day_index)
            movie_df = df.drop(df.index[day_index:]) # by 진성  
            movie_dfs2.append(movie_df)
            break

                

# 상영종료일 출력 대신 day_index 이후로 삭제.
# df = df.drop(df.index[day_index:])

# ==========================================================


def movie_start(df, i): # 적용할 데이터프레임, 영화 개봉일자를 가져올 movie_info를 위한 i 

    for j in range(len(df.index)):
        obj_day = df.iloc[j, 0] # 반복문으로 날짜 datetime 가져온다.

        if obj_day.strftime('%Y-%m-%d') == movie_info.iloc[i,1]: #문자열로 변환, 비교
            
            print(obj_day.strftime('%Y-%m-%d'))
            
            df = df.drop(df.index[:j])
            df = df.reset_index(drop=True)
            movie_dfs1.append(df)
            print(len(movie_dfs1))
            break
        
# ==========================================================

# 엑셀 파일 만큼 반복돌아서 DataFrame 생성
# 엑셀에서 생긴 첫 df리스트
dfs = []

for i in range(41): 
    
    movie_df = pd.read_excel(f'C:/xlsx/KOBIS_일자별_통계정보_2023-04-06 ({i}).xlsx',
                                        sheet_name = f'KOBIS_일자별_통계정보_2023-04-06 ({i})', 
                                        header = 3, thousands = ',')
    dfs.append(movie_df)
print('엑셀 파일 만큼 반복돌아서 DataFrame 생성 완료')    
# ==========================================================

# 영화제목을 파일 제목으로 하기위해 :를 -로 교체.
for i in range(41):
    movie_info.iloc[i,0]=movie_info.iloc[i,0].replace(":", "-")

print('영화제목을 파일 제목으로 하기위해 :를 -로 교체. end')
# ==========================================================

#영화 상영 시작일 함수 적용.
# dfs 에서 df로 가져오고 i이용해서 특정영화 개봉일 가져오기.

for i in range(41):
    df = dfs[i]
    movie_start(df,i)
    

print('영화 상영 시작일 함수 적용 end.')    
# ==========================================================

#영화 상영 종료일 함수 적용.

for i in range(41):
    df = movie_dfs1[i]    
    movie_end(df)
    print(i)
# 39, 38, 32, 28, 24, 21, 6, 5
# 21--
# ==========================================================

# 영화 상영 종료일로 자르고 저장한 리스트에서 가져와서 저장.
for i in range(41):
    df2 = movie_dfs2[i]
    #movie_end(df)
    df2.to_csv(f'C:/GitTest/10M_Movie/movie_DB/movie_1to41/{movie_info.iloc[i,0]}.csv')
    #print(i)


# 보완점

## 처음 0~40 (41)위까지 크롤링.


### 1.빠진 실미도, 태극기 추가

### 2. 천만영화 29개 대조군 800만인 41위 스파이더맨 ->

### 2배인 58위 까지 확장.

### 42위 ~ 58위 필요

In [ ]:
# 42위 ~ 58위 필요
# index 43~59